# Lec 32-34: Concurrent Programming Paradigm

Jianwen Zhu <jzhu@eecg.toronto.edu>
v2.0, 2024-09

Let's build an operating system for fun!

Disclaimer: Much material of this lecture is based on code samples from https://github.com/cl0ne/dabeaz-coroutines, originally by David Beazley.

## Overview 

* The Task Concept
    - In concurrent programming, one typically subdivides problems into "tasks"
    - Tasks have a few essential features
        - Independent control flow
        - Internal state
        - Can be scheduled (suspended/resumed)
        - Can communicate with other tasks

```
@coroutine
def grep(pattern):
    print( "Looking for %s" % pattern )
    while True:
        line = yield
        if pattern in line:
            print( line )
          
```

* Are Coroutines Tasks?
    - Let's look at the essential parts
    - Coroutines have their own control flow
      - A coroutine is just a sequence of statements like any other Python function
    - Coroutines have their internal own state
        - The locals live as long as the coroutine is active
        - They establish an execution environment
    - Coroutines can communicate
        - The .send() method sends data to a coroutine
        - yield expressions receive input
    -  Coroutines can be suspended and resumed
        - yield suspends execution
        - send() resumes execution
        - close() terminates execution

* Question: Can you perform multitasking using nothing but coroutines?

* Multi-Tasking System
    - Provide an illusion where mutliple, independant programs own the physical CPU
    - Context switch: suspending one task, and switch to another
    - By interrupt: hardware initiated (preemptive)
    - By trap: sofware initiated (cooperative)
      
* An Insight
    - The yield statement is a kind of "trap"
    - When a coroutine hits a "yield" statement, it immediately suspends execution
    - Control is passed back to whatever code made the generator function run (unseen)
    - If you treat yield as a trap, you can build a multitasking "operating system"--all in Python!

* Not Just for Fun
    - Non-blocking and asynchronous I/O
    - Example: servers capable of supporting thousands of simultaneous client connections
    - A lot of work has focused on event-driven systems or the "Reactor Model" (e.g.,Twisted)

  

## First Attempt: Interleaved Execution

* Step 1: Defining Tasks
  - A task is a wrapper around a coroutine 
  - There is only one operation : run()

In [ ]:
class Task(object):
    taskid = 0
    def __init__(self,target):
        Task.taskid += 1
        self.tid     = Task.taskid # Task ID
        self.target  = target # Target coroutine
        self.sendval = None   # Value to send

    def run(self):
        return self.target.send(self.sendval)

In [ ]:
def foo():
    print( "Part 1" )
    yield
    print( "Part 2" )
    yield

In [ ]:
t1 = Task(foo())
t1.run()

In [ ]:
t1.run()

* Step 2: Defining Scheduler

In [ ]:
from queue import Queue

class Scheduler(object):
        def __init__(self):
            self.ready   = Queue()
            self.taskmap = {}
            
        def new(self,target):
            newtask = Task(target)
            self.taskmap[newtask.tid] = newtask
            self.schedule(newtask)
            return newtask.tid
            
        def schedule(self,task):
            self.ready.put(task)
            
        def mainloop(self):
            i = 0 # avoiding running forever in Jupyter, do not really need it
            while self.taskmap and i < 100 :
                task = self.ready.get()
                result = task.run()
                self.schedule(task)
                i = i + 1 # not necessary

In [ ]:
def foo():
    while True:
        print( "I'm foo" )
        yield

def bar():
    while True:
        print( "I'm bar" )
        yield

In [ ]:
sched = Scheduler()
sched.new(foo())
sched.new(bar())
sched.mainloop()

## Second Attempt: Adding Temination

In [ ]:
def foo():
    for i in range(10):
        print( "I'm foo" )
        yield

In [ ]:
sched = Scheduler()
sched.new(foo())
sched.new(bar())
sched.mainloop()

In [ ]:
from queue import Queue


class Scheduler(object):
    def __init__(self):
        self.ready = Queue()
        self.taskmap = {}

    def new(self, target):
        newtask = Task(target)
        self.taskmap[newtask.tid] = newtask
        self.schedule(newtask)
        return newtask.tid

    def exit(self, task):
        print("Task %d terminated" % task.tid)
        del self.taskmap[task.tid]

    def schedule(self, task):
        self.ready.put(task)

    def mainloop(self):
        while self.taskmap:
            task = self.ready.get()
            try:
                result = task.run()
                self.schedule(task)
            except StopIteration:
                self.exit(task)

In [ ]:
def foo():
    for i in range(10):
        print("I'm foo")
        yield

def bar():
    for i in range(5):
        print("I'm bar")
        yield

sched = Scheduler()
sched.new(foo())
sched.new(bar())
sched.mainloop()

## Third Attempt: Adding System Call

* System Call is the *interface* of an OS 
    - In a real operating system, traps are how application programs request the services of the operating system (syscalls)
    - In our code, the scheduler is the operating system and the yield statement is a trap
    - To request the service of the scheduler, tasks will use the yield statement with a value

In [7]:
# ------------------------------------------------------------
#                       === Tasks ===
# ------------------------------------------------------------
class Task(object):
    taskid = 0

    def __init__(self, target):
        Task.taskid += 1
        self.tid = Task.taskid  # Task ID
        self.target = target  # Target coroutine
        self.sendval = None  # Value to send

    # Run a task until it hits the next yield statement
    def run(self):
        return self.target.send(self.sendval)


# ------------------------------------------------------------
#                      === Scheduler ===
# ------------------------------------------------------------
from queue import Queue


class Scheduler(object):
    def __init__(self):
        self.ready = Queue()
        self.taskmap = {}

    def new(self, target):
        newtask = Task(target)
        self.taskmap[newtask.tid] = newtask
        self.schedule(newtask)
        return newtask.tid

    def exit(self, task):
        print("Task %d terminated" % task.tid)
        del self.taskmap[task.tid]

    def schedule(self, task):
        self.ready.put(task)

    def mainloop(self):
        while self.taskmap:
            task = self.ready.get()
            try:
                result = task.run()
                if isinstance(result, SystemCall):
                    result.task = task
                    result.sched = self
                    result.handle()
                    continue
                self.schedule(task)
            except StopIteration:
                self.exit(task)


* Scheduler modification:
  ```
  ...
                result = task.run()
                if isinstance(result, SystemCall):
                    result.task = task
                    result.sched = self
                    result.handle()
                    continue
  ...
  ```
  task.run() returns an object, which is the value of yield, and in our case, would be an SystemCall object. We will simply call the handler() method of the SystemCall object.
  
* Implementing your first system call
  - getpid !
  - Recall the implementation of Task.run()

```
 class Task(object):
        ...
        def run(self):
            return self.target.send(self.sendval)
```
  -  The sendval attribute of a task is like a return value from a system call. It's value is sent into the task when it runs again.

In [6]:
class SystemCall(object):
    def handle(self):
        pass

# Return a task's ID number
class GetTid(SystemCall):
    def handle(self):
        self.task.sendval = self.task.tid     # this would be return value of yield
        self.sched.schedule(self.task)        # put myself back into ready queue

* Let's try it out!

In [8]:
def foo():
        mytid = yield GetTid()
        for i in range(5):
            print("I'm foo", mytid)
            yield

def bar():
        mytid = yield GetTid()
        for i in range(10):
            print("I'm bar", mytid)
            yield

sched = Scheduler()
sched.new(foo())
sched.new(bar())
sched.mainloop()

I'm foo 1
I'm bar 2
I'm foo 1
I'm bar 2
I'm foo 1
I'm bar 2
I'm foo 1
I'm bar 2
I'm foo 1
I'm bar 2
Task 1 terminated
I'm bar 2
I'm bar 2
I'm bar 2
I'm bar 2
I'm bar 2
Task 2 terminated


## Fifth Attempt: Task Management

* Desired
  - Real operating systems have a strong notion of "protection" (e.g., memory protection)
  - Application programs are not strongly linked to the OS kernel (traps are only interface)
  - For sanity, we are going to emulate this
      - Tasks do not see the scheduler
      - Tasks do not see other tasks
      - yield is the only external interface

* Task Management
    - Let's make more some system calls
    - Some task management functions
        - Create a new task
        - Kill an existing task
        - Wait for a task to exit
    - These mimic common operations with threads or processes

In [9]:
# ------------------------------------------------------------
#                       === Tasks ===
# ------------------------------------------------------------
class Task(object):
    taskid = 0

    def __init__(self, target):
        Task.taskid += 1
        self.tid = Task.taskid  # Task ID
        self.target = target  # Target coroutine
        self.sendval = None  # Value to send

    # Run a task until it hits the next yield statement
    def run(self):
        return self.target.send(self.sendval)


# ------------------------------------------------------------
#                      === Scheduler ===
# ------------------------------------------------------------
from queue import Queue


class Scheduler(object):
    def __init__(self):
        self.ready = Queue()
        self.taskmap = {}

    def new(self, target):
        newtask = Task(target)
        self.taskmap[newtask.tid] = newtask
        self.schedule(newtask)
        return newtask.tid
    
    def exit(self, task):
        print("Task %d terminated" % task.tid)
        del self.taskmap[task.tid]

    def schedule(self, task):
        self.ready.put(task)

    def mainloop(self):
        while self.taskmap:
            task = self.ready.get()
            try:
                result = task.run()
                if isinstance(result, SystemCall):
                    result.task = task
                    result.sched = self
                    result.handle()
                    continue
                self.schedule(task)
            except StopIteration:
                self.exit(task)




In [10]:
# ------------------------------------------------------------
#                   === System Calls ===
# ------------------------------------------------------------
class SystemCall(object):
    def handle(self):
        pass


# Return a task's ID number
class GetTid(SystemCall):
    def handle(self):
        self.task.sendval = self.task.tid
        self.sched.schedule(self.task)

# Create a new task
class NewTask(SystemCall):
    def __init__(self, target):
        self.target = target

    def handle(self):
        tid = self.sched.new(self.target)
        self.task.sendval = tid
        self.sched.schedule(self.task)


# Kill a task
class KillTask(SystemCall):
    def __init__(self, tid):
        self.tid = tid

    def handle(self):
        task = self.sched.taskmap.get(self.tid, None)
        if task:
            task.target.close()
            self.task.sendval = True
        else:
            self.task.sendval = False
        self.sched.schedule(self.task)



Equipped with the new system calls, tasks themselves have the capability of starting or killing other tasks, much the same way as in the real OS!

* An Example

In [11]:
def foo():
        mytid = yield GetTid()
        while True:
            print("I'm foo", mytid)
            yield

def main():
        child = yield NewTask(foo())  # Launch new task
        for i in range(5):
            yield
        yield KillTask(child)  # Kill the task
        print("main done")

sched = Scheduler()
sched.new(main())
sched.mainloop()

I'm foo 2
I'm foo 2
I'm foo 2
I'm foo 2
I'm foo 2
Task 2 terminated
main done
Task 1 terminated


## The Sixth Attempt: Blocking Wait

The above code is still a bit wiered. In main(), we have
```
        for i in range(5):
            yield

```
really just to allow the child task foo() to complete. We need a better scheme!

* Waiting for a task: A better looking code should be the following: 
 
```
def foo():
    for i in xrange(5):
        print( "I'm foo" )
        yield

def main():
    child = yield NewTask(foo())
    print( "Waiting for child" )
    yield WaitTask(child)
    print( "Child done" )
```

    - The task that waits has to remove itself from the run queue--it sleeps until child exits
    - This requires some scheduler changes and new system call.

In [12]:
# ------------------------------------------------------------
#                       === Tasks ===
# ------------------------------------------------------------
class Task(object):
    taskid = 0

    def __init__(self, target):
        Task.taskid += 1
        self.tid = Task.taskid  # Task ID
        self.target = target  # Target coroutine
        self.sendval = None  # Value to send

    # Run a task until it hits the next yield statement
    def run(self):
        return self.target.send(self.sendval)


# ------------------------------------------------------------
#                      === Scheduler ===
# ------------------------------------------------------------
from queue import Queue

class Scheduler(object):
    def __init__(self):
        self.ready = Queue()
        self.taskmap = {}

        # Tasks waiting for other tasks to exit
        self.exit_waiting = {}         ## NEW CODE HERE

    def new(self, target):
        newtask = Task(target)
        self.taskmap[newtask.tid] = newtask
        self.schedule(newtask)
        return newtask.tid

#### NEW CODE BEGIN
    def exit(self, task):
        print("Task %d terminated" % task.tid)
        del self.taskmap[task.tid]
        # Notify other tasks waiting for exit
        for task in self.exit_waiting.pop(task.tid, []):
            self.schedule(task)

    def waitforexit(self, task, waittid):
        if waittid in self.taskmap:
            self.exit_waiting.setdefault(waittid, []).append(task)
            return True
        return False
### NEW CODE END

    def schedule(self, task):
        self.ready.put(task)
        
    def mainloop(self):
        while self.taskmap:
            task = self.ready.get()
            try:
                result = task.run()
                if isinstance(result, SystemCall):
                    result.task = task
                    result.sched = self
                    result.handle()
                    continue
                self.schedule(task)
            except StopIteration:
                self.exit(task)


In [13]:
# ------------------------------------------------------------
#                   === System Calls ===
# ------------------------------------------------------------
class SystemCall(object):
    def handle(self):
        pass


# Return a task's ID number
class GetTid(SystemCall):
    def handle(self):
        self.task.sendval = self.task.tid
        self.sched.schedule(self.task)


# Create a new task
class NewTask(SystemCall):
    def __init__(self, target):
        self.target = target

    def handle(self):
        tid = self.sched.new(self.target)
        self.task.sendval = tid
        self.sched.schedule(self.task)


# Kill a task
class KillTask(SystemCall):
    def __init__(self, tid):
        self.tid = tid

    def handle(self):
        task = self.sched.taskmap.get(self.tid, None)
        if task:
            task.target.close()
            self.task.sendval = True
        else:
            self.task.sendval = False
        self.sched.schedule(self.task)



In [14]:
# Wait for a task to exit
class WaitTask(SystemCall):
    def __init__(self, tid):
        self.tid = tid

    def handle(self):
        result = self.sched.waitforexit(self.task, self.tid)
        self.task.sendval = result
        # If waiting for a non-existent task,
        # return immediately without waiting
        if not result:
            self.sched.schedule(self.task)

* An Example

In [16]:
def foo():
        for i in range(5):
            print("I'm foo")
            yield

def main():
        child = yield NewTask(foo())
        print("Waiting for child")
        yield WaitTask(child)
        print("Child done")

sched = Scheduler()
sched.new(main())
sched.mainloop()

I'm foo
Waiting for child
I'm foo
I'm foo
I'm foo
I'm foo
Task 4 terminated
Child done
Task 3 terminated


* Summary
    - The only way for tasks to refer to other tasks is using the integer task ID assigned by the the scheduler
    - This is an encapsulation and safety strategy
    - It keeps tasks separated (no linking to internals)
    - It places all task management in the scheduler (which is where it properly belongs)

## (Bad) Echo Server

Now let's try to do something close to real: develop a TCP server program!

In [ ]:
from socket import *

def handle_client(client, addr):
    print("Connection from", addr)
    while True:
        data = client.recv(65536)
        if not data:
            break
        client.send(data)
    client.close()
    print("Client closed")
    yield  # Make the function a generator/coroutine


def server(port):
    print("Server starting")
    sock = socket(AF_INET, SOCK_STREAM)
    sock.bind(("", port))
    sock.listen(5)
    while True:
        client, addr = sock.accept()
        yield NewTask(handle_client(client, addr))


def alive():
    while True:
        print("I'm alive!")
        yield

In [ ]:
# DO NOT RUN THIS, IT WILL FREEZE!
sched = Scheduler()
sched.new(alive())
sched.new(server(45000))
sched.mainloop()

* Blocking Operations
   - In the example various I/O operations block
 
    ``` 
            client,addr = sock.accept()
            data = client.recv(65536)
            client.send(data)
    ```
     
    - The real operating system (e.g., Linux) suspends the entire Python interpreter until the I/O operation completes
      
    - Clearly this is pretty undesirable for our multitasking operating system (any blocking operation freezes the whole program)

* Non-blocking I/O
    - The select module can be used to monitor a collection of sockets (or files) for activity
    ```
    reading = []    # List of sockets waiting for read
    writing = []    # List of sockets waiting for write
    # Poll for I/O activity
    r,w,e = select.select(reading,writing,[],timeout)
    # r is list of sockets with incoming data
    # w is list of sockets ready to accept outgoing data
    # e is list of sockets with an error state
    ```
    - This can be used to add I/O support to our OS
    - This is going to be similar to task waiting

## Seventh Attempt: Non-Blocking IO

In [20]:
# ------------------------------------------------------------
#                       === Tasks ===
# ------------------------------------------------------------
class Task(object):
    taskid = 0

    def __init__(self, target):
        Task.taskid += 1
        self.tid = Task.taskid  # Task ID
        self.target = target  # Target coroutine
        self.sendval = None  # Value to send

    # Run a task until it hits the next yield statement
    def run(self):
        return self.target.send(self.sendval)

# ------------------------------------------------------------
#                      === Scheduler ===
# ------------------------------------------------------------
from queue import Queue
import select

class Scheduler(object):
    def __init__(self):
        self.ready = Queue()
        self.taskmap = {}

        # Tasks waiting for other tasks to exit
        self.exit_waiting = {}

#### NEW BEGIN
        # I/O waiting
        self.read_waiting = {}             # New Here
        self.write_waiting = {}            # New Here
#### NEW END

    def new(self, target):
        newtask = Task(target)
        self.taskmap[newtask.tid] = newtask
        self.schedule(newtask)
        return newtask.tid

    def exit(self, task):
        print("Task %d terminated" % task.tid)
        del self.taskmap[task.tid]
        # Notify other tasks waiting for exit
        for task in self.exit_waiting.pop(task.tid, []):
            self.schedule(task)

    def waitforexit(self, task, waittid):
        if waittid in self.taskmap:
            self.exit_waiting.setdefault(waittid, []).append(task)
            return True
        else:
            return False

#### NEW BEGIN
    # I/O waiting
    def waitforread(self, task, fd):
        self.read_waiting[fd] = task

    def waitforwrite(self, task, fd):
        self.write_waiting[fd] = task

    def iopoll(self, timeout):
        if self.read_waiting or self.write_waiting:
            r, w, e = select.select(self.read_waiting, self.write_waiting, [], timeout)
            for fd in r:
                self.schedule(self.read_waiting.pop(fd))
            for fd in w:
                self.schedule(self.write_waiting.pop(fd))

    def iotask(self):
        while True:
            if self.ready.empty():
                self.iopoll(None)
            else:
                self.iopoll(0)
            yield
#### NEW END

    def schedule(self, task):
        self.ready.put(task)

    def mainloop(self):
        self.new(self.iotask())
        while self.taskmap:
            task = self.ready.get()
            try:
                result = task.run()
                if isinstance(result, SystemCall):
                    result.task = task
                    result.sched = self
                    result.handle()
                    continue
                self.schedule(task)
            except StopIteration:
                self.exit(task)


In [17]:

# ------------------------------------------------------------
#                   === System Calls ===
# ------------------------------------------------------------


class SystemCall(object):
    def handle(self):
        pass


# Return a task's ID number
class GetTid(SystemCall):
    def handle(self):
        self.task.sendval = self.task.tid
        self.sched.schedule(self.task)


# Create a new task
class NewTask(SystemCall):
    def __init__(self, target):
        self.target = target

    def handle(self):
        tid = self.sched.new(self.target)
        self.task.sendval = tid
        self.sched.schedule(self.task)

# Kill a task
class KillTask(SystemCall):
    def __init__(self, tid):
        self.tid = tid

    def handle(self):
        task = self.sched.taskmap.get(self.tid, None)
        if task:
            task.target.close()
            self.task.sendval = True
        else:
            self.task.sendval = False
        self.sched.schedule(self.task)


# Wait for a task to exit
class WaitTask(SystemCall):
    def __init__(self, tid):
        self.tid = tid

    def handle(self):
        result = self.sched.waitforexit(self.task, self.tid)
        self.task.sendval = result
        # If waiting for a non-existent task,
        # return immediately without waiting
        if not result:
            self.sched.schedule(self.task)


# Wait for reading
class ReadWait(SystemCall):
    def __init__(self, f):
        self.f = f

    def handle(self):
        fd = self.f.fileno()
        self.sched.waitforread(self.task, fd)



# Wait for writing
class WriteWait(SystemCall):
    def __init__(self, f):
        self.f = f

    def handle(self):
        fd = self.f.fileno()
        self.sched.waitforwrite(self.task, fd)


* A New Echo Server

In [18]:
from socket import *

def handle_client(client, addr):
    print("Connection from", addr)
    while True:
        yield ReadWait(client)
        data = client.recv(65536)
        if not data:
            break
        yield WriteWait(client)
        client.send(data)
    client.close()
    print("Client closed")


def server(port):
    print("Server starting")
    sock = socket(AF_INET, SOCK_STREAM)
    sock.setsockopt(SOL_SOCKET, SO_REUSEADDR, 1)
    sock.setblocking(0)
    sock.bind(("", port))
    sock.listen(5)
    while True:
        yield ReadWait(sock)
        client, addr = sock.accept()
        yield NewTask(handle_client(client, addr))

In [21]:
import threading

def alive():
    while True:
        print("I'm alive!")
        yield

def run_me():
    sched = Scheduler()
    # sched.new(alive())
    sched.new(server(45000))
    sched.mainloop()

thread = threading.Thread(target=run_me)
thread.start()

Server starting
Connection from ('127.0.0.1', 54632)
Connection from ('127.0.0.1', 43680)
Client closed
Task 3 terminated
Client closed
Task 4 terminated
Connection from ('127.0.0.1', 37500)
Client closed
Task 5 terminated
Connection from ('127.0.0.1', 54602)
Client closed
Task 6 terminated
Connection from ('127.0.0.1', 38772)
Client closed
Task 7 terminated
Connection from ('127.0.0.1', 57106)
Client closed
Task 8 terminated
Connection from ('127.0.0.1', 44638)
Client closed
Task 9 terminated
Connection from ('127.0.0.1', 58324)
Client closed
Task 10 terminated
Connection from ('127.0.0.1', 55630)
Client closed
Task 11 terminated
Connection from ('127.0.0.1', 32966)
Client closed
Task 12 terminated
Connection from ('127.0.0.1', 52000)
Client closed
Task 13 terminated
Connection from ('127.0.0.1', 50092)
Client closed
Task 14 terminated
Connection from ('127.0.0.1', 38218)
Client closed
Task 15 terminated
Connection from ('127.0.0.1', 49366)
Client closed
Task 16 terminated
Connection 

## Recap

What have we learned? A Lot!

* For Fun: Full-blown Multitask OS
    - Coroutines are task!
    - Yields are system calls
    
* For Profit: Extremely lightweight
    - Coroutine are *cheap* state machines in disguise
    - Event-driven programming with non-blocking
    - But without the programming burden of event-driven programming
    - Solve C10K performance problem of server programming
    - HAVE YOUR CAKE AND EAT IT TOO!